In [ ]:
!pip install "polars[all]"
!pip install seaborn
!pip install scikit-learn

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, f1_score, accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from statistics import mean
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

sb.set()
pd.set_option('display.max_columns', None)

In [ ]:
import os, datetime
import numpy as np
import polars as pl
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error
import pandas as pd

# Settings
work_dir = "C:\\Users\\harim\\Downloads\\Hackathon"

ret_var = "stock_ret"
start_date = datetime.date(2005, 1, 1)
end_date   = datetime.date(2026, 1, 1)

print("Imports and settings loaded at", datetime.datetime.now())


In [ ]:
fac_path = os.path.join(work_dir, "factor_char_list.csv")
stock_vars = pl.read_csv(fac_path)["variable"].to_list()
print("Loaded predictors:", len(stock_vars))
print(stock_vars[:10])  # preview first 10


In [ ]:
csv_path = os.path.join(work_dir, "ret_sample.csv")
raw = pl.read_csv(
    csv_path,
    try_parse_dates=True,
    schema_overrides={
        **{v: pl.Float32 for v in stock_vars},  # predictors as floats
        "gvkey": pl.Utf8, "iid": pl.Utf8, "id": pl.Utf8,
    },
    low_memory=True,
    rechunk=True,
)

# Add predictor date (char_date as YYYYMMDD -> pl.Date)
raw = raw.with_columns(
    pl.col("char_date").cast(pl.Utf8).str.strptime(pl.Date, "%Y%m%d").alias("date")
)

raw = raw.filter(pl.col(ret_var).is_not_null())
print("Raw shape:", raw.shape)
print(raw.select(["date", "ret_eom", ret_var]).head(5))


In [ ]:
def rank_to_unit(df: pl.DataFrame) -> pl.DataFrame:
    out = df
    for var in stock_vars:
        median = out[var].median()
        out = out.with_columns(
            pl.when(pl.col(var).is_null()).then(pl.lit(median, dtype=pl.Float32)).otherwise(pl.col(var)).alias(var)
        )
        out = out.with_columns((pl.col(var).rank("dense") - 1).cast(pl.Float32).alias(var))
        maxv = out[var].max()
        if maxv is None or maxv == 0:
            out = out.with_columns(pl.lit(0.0, dtype=pl.Float32).alias(var))
        else:
            out = out.with_columns(((pl.col(var) / pl.lit(maxv, dtype=pl.Float32)) * 2.0 - 1.0).alias(var))
    return out

data = raw.group_by("date", maintain_order=True).map_groups(rank_to_unit)
print("Rank-transformed shape:", data.shape)
print(data.head(5))


In [ ]:
import polars as pl
import os

# ✅ Choose top predictors (replace with your chosen subset)
selected_predictors = ["chcsho_12m","rd_me", "at_me", "fcf_me", "ope_be", "turnover_126d", "eqnpo_12m", "rd_sale", "bidaskhl_21d"]  # example
columns_to_keep = ["date", "gvkey", "iid", "id", ret_var] + selected_predictors

# ✅ Keep only chosen columns
cleaned = raw.select([c for c in columns_to_keep if c in raw.columns])

# ✅ Drop rows with nulls in these columns
cleaned = cleaned.drop_nulls()

# ✅ Save cleaned dataset to a new CSV
output_path = os.path.join(work_dir, "cleaned_data.csv")
cleaned.write_csv(output_path)

print("Cleaned data shape:", cleaned.shape)
print("Saved cleaned file to:", output_path)
print(cleaned.head(10))


In [ ]:

pred_frames = []
counter = 0

max_data_date = data.select(pl.col("date").max()).item()
safe_end = min(end_date, (max_data_date - datetime.timedelta(days=365)))

while (start_date + datetime.timedelta(days=365*(11 + counter))) <= safe_end:
    cutoff = [
        start_date,
        start_date + datetime.timedelta(days=365*(8  + counter)),
        start_date + datetime.timedelta(days=365*(10 + counter)),
        start_date + datetime.timedelta(days=365*(11 + counter)),
    ]
    print("Window", counter, cutoff)

    train = data.filter((pl.col("date") >= cutoff[0]) & (pl.col("date") < cutoff[1]))
    val   = data.filter((pl.col("date") >= cutoff[1]) & (pl.col("date") < cutoff[2]))
    test  = data.filter((pl.col("date") >= cutoff[2]) & (pl.col("date") < cutoff[3]))

    if train.height == 0 or val.height == 0 or test.height == 0:
        print("Empty window, skipping")
        counter += 1
        continue

    # Convert to numpy
    X_train = train.select(stock_vars).to_numpy()
    Y_train = train.select(ret_var).to_numpy().ravel()
    X_val   = val.select(stock_vars).to_numpy()
    Y_val   = val.select(ret_var).to_numpy().ravel()
    X_test  = test.select(stock_vars).to_numpy()
    Y_test  = test.select(ret_var).to_numpy().ravel()

    # Standardize
    scaler = StandardScaler().fit(X_train)
    X_train, X_val, X_test = scaler.transform(X_train), scaler.transform(X_val), scaler.transform(X_test)

    Y_mean = Y_train.mean()
    Y_dm   = Y_train - Y_mean

    reg_pred = test.select(["year", "month", "ret_eom", "id", ret_var]).to_pandas()

    # Models
    reg = LinearRegression(fit_intercept=False).fit(X_train, Y_dm)
    reg_pred["ols"] = reg.predict(X_test) + Y_mean

    # lambdas = np.arange(-4, 4.1, 0.1)
    # val_mse = [mean_squared_error(Y_val, Lasso(alpha=10**p, max_iter=1_000_000, fit_intercept=False).fit(X_train, Y_dm).predict(X_val) + Y_mean) for p in lambdas]
    # best = lambdas[int(np.argmin(val_mse))]
    # reg_pred["lasso"] = Lasso(alpha=10**best, max_iter=1_000_000, fit_intercept=False).fit(X_train, Y_dm).predict(X_test) + Y_mean

    lambdas = np.arange(-1, 8.1, 0.1)
    val_mse = [mean_squared_error(Y_val, Ridge(alpha=(10**p)*0.5, fit_intercept=False).fit(X_train, Y_dm).predict(X_val) + Y_mean) for p in lambdas]
    best = lambdas[int(np.argmin(val_mse))]
    reg_pred["ridge"] = Ridge(alpha=(10**best)*0.5, fit_intercept=False).fit(X_train, Y_dm).predict(X_test) + Y_mean

    # lambdas = np.arange(-4, 4.1, 0.1)
    # val_mse = [mean_squared_error(Y_val, ElasticNet(alpha=10**p, max_iter=1_000_000, fit_intercept=False).fit(X_train, Y_dm).predict(X_val) + Y_mean) for p in lambdas]
    # best = lambdas[int(np.argmin(val_mse))]
    # reg_pred["en"] = ElasticNet(alpha=10**best, max_iter=1_000_000, fit_intercept=False).fit(X_train, Y_dm).predict(X_test) + Y_mean

    pred_frames.append(reg_pred)
    counter += 1

print("Finished backtest loop with", counter, "windows")


In [ ]:
# ✅ Load the cleaned CSV created earlier
csv_path = os.path.join(work_dir, "cleaned_data.csv")
df = pd.read_csv(csv_path, parse_dates=["date"])

# -------------------------------
# 1. Summary Stats
# -------------------------------
print("\n=== Summary Statistics ===")
print(df.describe(include="all"))

print("\n=== Missing Values ===")
print(df.isnull().sum().sort_values(ascending=False))

# -------------------------------
# 2. Distribution of Stock Returns
# -------------------------------
plt.figure(figsize=(8,6))
sns.histplot(df[ret_var], bins=50, kde=True)
plt.title("Distribution of Stock Returns")
plt.xlabel("Returns")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

# -------------------------------
# 3. Correlation Heatmap
# -------------------------------
predictors = [col for col in df.columns if col not in ["date", "gvkey", "iid", "id", ret_var]]
plt.figure(figsize=(12,8))
corr = df[predictors + [ret_var]].corr()
sns.heatmap(corr, annot=False, cmap="coolwarm", center=0)
plt.title("Correlation Heatmap (Predictors vs Return)")
plt.show()

# -------------------------------
# 4. Top Correlated Predictors
# -------------------------------
corr_with_return = corr[ret_var].drop(ret_var).sort_values(key=abs, ascending=False)
print("\n=== Top Predictors Correlated with Return ===")
print(corr_with_return.head(10))

# -------------------------------
# 5. Pairplot (Predictors vs Return)
# -------------------------------
sns.pairplot(df[[ret_var] + predictors[:4]], diag_kind="kde")  # first few vars
plt.suptitle("Pairwise Plots (Returns vs Predictors)", y=1.02)
plt.show()

# -------------------------------
# 6. Time Series of Returns
# -------------------------------
plt.figure(figsize=(12,6))
df.groupby("date")[ret_var].mean().plot()
plt.title("Average Stock Return Over Time")
plt.ylabel("Return")
plt.grid(True)
plt.show()

# -------------------------------
# 7. Scatter Plots for Key Predictors
# -------------------------------
top_vars = corr_with_return.head(3).index.tolist()  # top 3 predictors
for v in top_vars:
    plt.figure(figsize=(8,6))
    sns.scatterplot(x=df[v], y=df[ret_var], alpha=0.5)
    plt.title(f"{v} vs Stock Return")
    plt.xlabel(v)
    plt.ylabel("Return")
    plt.grid(True)
    plt.show()

# -------------------------------
# 8. Predictive Modeling (OLS, Ridge, etc.)
# -------------------------------

# Extract predictors (X) and target (y)
X = df[predictors].values
y = df[ret_var].values

# Standardize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into train/test set
train_size = int(0.8 * len(df))
X_train, X_test = X_scaled[:train_size], X_scaled[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# 1. OLS
ols_model = LinearRegression(fit_intercept=False).fit(X_train, y_train)
ols_pred = ols_model.predict(X_test)
ols_r2 = 1 - np.sum((y_test - ols_pred)**2) / np.sum((y_test - y_test.mean())**2)
print("\nOLS Model R²:", ols_r2)

# 2. Ridge Regression (hyperparameter tuning)
ridge_model = Ridge(alpha=1.0).fit(X_train, y_train)
ridge_pred = ridge_model.predict(X_test)
ridge_r2 = 1 - np.sum((y_test - ridge_pred)**2) / np.sum((y_test - y_test.mean())**2)
print("Ridge Model R²:", ridge_r2)

# 3. Lasso Regression (hyperparameter tuning)
lasso_model = Lasso(alpha=0.1).fit(X_train, y_train)
lasso_pred = lasso_model.predict(X_test)
lasso_r2 = 1 - np.sum((y_test - lasso_pred)**2) / np.sum((y_test - y_test.mean())**2)
print("Lasso Model R²:", lasso_r2)

# 4. ElasticNet Regression (hyperparameter tuning)
en_model = ElasticNet(alpha=0.1, l1_ratio=0.5).fit(X_train, y_train)
en_pred = en_model.predict(X_test)
en_r2 = 1 - np.sum((y_test - en_pred)**2) / np.sum((y_test - y_test.mean())**2)
print("ElasticNet Model R²:", en_r2)
